# **Lunar Lander con Q-Learning**

### **1. Bibliotecas**

In [1]:
import sys
# Tal vez tengan que ejecutar lo siguiente en sus máquinas (ubuntu 20.04)
# sudo apt-get remove swig
# sudo apt-get install swig3.0
# sudo ln -s /usr/bin/swig3.0 /usr/bin/swig
# En windows tambien puede ser necesario MSVC++
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random

### **2. Jugando a mano**

A continuación se puede jugar un episodio del lunar lander. Se controlan los motores con el teclado. Notar que solo se puede realizar una acción a la vez (que es parte del problema), y que en esta implementación, izq toma precedencia sobre derecha, que toma precedencia sobre el motor principal.

In [2]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)

import pygame
from pygame.locals import *

# Inicializar pygame (para el control con el teclado) y el ambiente
pygame.init()
env = gym.make('LunarLander-v2', render_mode='human')
env.reset()
pygame.display.set_caption('Lunar Lander')

clock = pygame.time.Clock()
done = False

while not done:
    for event in pygame.event.get():
        if event.type == QUIT:
            done = True
            break

    keys = pygame.key.get_pressed()

    # Map keys to actions
    if keys[K_LEFT]:
        action = 3  # Fire left orientation engine
    elif keys[K_RIGHT]:
        action = 1 # Fire right orientation engine
    elif keys[K_UP]:
        action = 2  # Fire main engine
    else:
        action = 0  # Do nothing

    _, _, terminated, truncated, _ = env.step(action)
    env.render()
    clock.tick(10)

    if terminated or truncated:
        done = True

env.close()
pygame.quit()

## **3. Discretizando el estado**

El estado consiste de posiciones y velocidades en (x,y,theta) y en información de contacto de los pies con la superficie.

Como varios de estos son continuos, tenemos que discretizarlos para aplicar nuestro algoritmo de aprendizaje por refuerzo tabular.

In [3]:
# Cuántos bins queremos por dimensión
# Pueden considerar variar este parámetro
bins_per_dim = 15

# Estado:
# (x, y, x_vel, y_vel, theta, theta_vel, pie_izq_en_contacto, pie_derecho_en_contacto)
NUM_BINS = [bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, bins_per_dim, 2, 2]

env = gym.make('LunarLander-v2')
env.reset()

# Tomamos los rangos del env
OBS_SPACE_HIGH = env.observation_space.high
OBS_SPACE_LOW = env.observation_space.low
OBS_SPACE_LOW[1] = 0 # Para la coordenada y (altura), no podemos ir más abajo que la zona dea aterrizae (que está en el 0, 0)

# Los bins para cada dimensión
bins = [
    np.linspace(OBS_SPACE_LOW[i], OBS_SPACE_HIGH[i], NUM_BINS[i] - 1)
    for i in range(len(NUM_BINS) - 2) # last two are binary
]
# Se recomienda observar los bins para entender su estructura
#print ("Bins: ", bins)
print("High: ", OBS_SPACE_HIGH)
print("Low: ", OBS_SPACE_LOW)
def discretize_state(state, bins):
    """Discretize the continuous state into a tuple of discrete indices."""
    state_disc = list()
    for i in range(len(state)):
        if i >= len(bins):  # For binary features (leg contacts)
            state_disc.append(int(state[i]))
        else:
            state_disc.append(
                np.digitize(state[i], bins[i])
            )
    return tuple(state_disc)

High:  [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ]
Low:  [-1.5        0.        -5.        -5.        -3.1415927 -5.
 -0.        -0.       ]


In [4]:
# Ejemplos
print(discretize_state([0.0, 0.0, 0, 0, 0, 0, 1, 1], bins)) # En la zona de aterrizaje y quieto
print(discretize_state([0, 1.5, 0, 0, 0, 0, 0, 0], bins)) # Comenzando la partida, arriba y en el centro

(np.int64(7), np.int64(1), np.int64(7), np.int64(7), np.int64(7), np.int64(7), 1, 1)
(np.int64(7), np.int64(14), np.int64(7), np.int64(7), np.int64(7), np.int64(7), 0, 0)


## **4. Agentes y la interacción con el entorno**

Vamos a definir una interfaz para nuestro agente:

In [5]:
class Agente:
    def elegir_accion(self, estado, max_accion, explorar = True) -> int:
        """Elegir la accion a tomar en el estado actual y el espacio de acciones
            - estado_anterior: el estado desde que se empezó
            - estado_siguiente: el estado al que se llegó
            - accion: la acción que llevo al agente desde estado_anterior a estado_siguiente
            - recompensa: la recompensa recibida en la transicion
            - terminado: si el episodio terminó
        """
        pass

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        """Aprender a partir de la tupla 
            - estado_anterior: el estado desde que se empezó
            - estado_siguiente: el estado al que se llegó
            - accion: la acción que llevo al agente desde estado_anterior a estado_siguiente
            - recompensa: la recompensa recibida en la transicion
            - terminado: si el episodio terminó en esta transición
        """
        pass

    def fin_episodio(self):
        """Actualizar estructuras al final de un episodio"""
        pass

Para un agente aleatorio, la implementación sería:

In [6]:
import random

class AgenteAleatorio(Agente):
    def elegir_accion(self, estado, max_accion, explorar = True) -> int:
        # Elige una acción al azar
        return random.randrange(max_accion)

    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        # No aprende
        pass

    def fin_episodio(self):
        # Nada que actualizar
        pass

Luego podemos definir una función para ejecutar un episodio con un agente dado:

In [7]:
def ejecutar_episodio(agente, aprender = True, render = None, max_iteraciones=500):
    entorno = gym.make('LunarLander-v2', render_mode=render).env
    
    iteraciones = 0
    recompensa_total = 0

    termino = False
    truncado = False
    estado_anterior, info = entorno.reset()
    while iteraciones < max_iteraciones and not termino and not truncado:
        # Le pedimos al agente que elija entre las posibles acciones (0..entorno.action_space.n)
        accion = agente.elegir_accion(estado_anterior, entorno.action_space.n, aprender)
        # Realizamos la accion
        estado_siguiente, recompensa, termino, truncado, info = entorno.step(accion)
        # Le informamos al agente para que aprenda
        if (aprender):
            agente.aprender(estado_anterior, estado_siguiente, accion, recompensa, termino)

        estado_anterior = estado_siguiente
        iteraciones += 1
        recompensa_total += recompensa
    if (aprender):
        agente.fin_episodio()
    entorno.close()
    return recompensa_total

In [8]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)

# Ejecutamos un episodio con el agente aleatorio y modo render 'human', para poder verlo
ejecutar_episodio(AgenteAleatorio(), render = 'human')

np.float64(-104.25733799682678)

Podemos ejecutar este ambiente muchas veces y tomar métricas al respecto

In [9]:
agente = AgenteAleatorio()
recompensa_episodios = []

exitos = 0
num_episodios = 100
for i in range(num_episodios):
    recompensa = ejecutar_episodio(agente)
    # Los episodios se consideran exitosos si se obutvo 200 o más de recompensa total
    if (recompensa >= 200):
        exitos += 1
    recompensa_episodios += [recompensa]

import numpy
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

Tasa de éxito: 0.0. Se obtuvo -169.90152325832003 de recompensa, en promedio


### **5. Programando un agente que aprende**

La tarea a realizar consiste en programar un agente de aprendizaje por refuerzos:

In [10]:
import numpy as np
import random
from collections import defaultdict
class AgenteRL(Agente):
    # Agregar código aqui

    # Pueden agregar parámetros al constructor
    def __init__(self, max_accion, bins, initial_epsilon: float, epsilon_decay: float, final_epsilon: float, discount_factor: float = 0.95):
        super().__init__()
        """Initialize a Reinforcement Learning agent with an empty dictionary
        of state-action values (q_values), a learning rate and an epsilon.

        Args:
            learning_rate: The learning rate
            initial_epsilon: The initial epsilon value
            epsilon_decay: The decay for epsilon
            final_epsilon: The final epsilon value
            discount_factor: The discount factor for computing the Q-value
        """
        self.q_table = defaultdict(lambda: np.zeros(max_accion))
        self.visitas = defaultdict(lambda: np.zeros(max_accion))

        self.discount_factor = discount_factor

        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        self.bins = bins

        self.training_error = []
    
    def elegir_accion(self, estado, max_accion, explorar = True) -> int:

        estado_discreto = discretize_state(estado, self.bins)
        if (explorar and np.random.random() < self.epsilon):
            return random.randrange(max_accion)
        else:
            return int(np.argmax(self.q_table[estado_discreto]))
    
    def aprender(self, estado_anterior, estado_siguiente, accion, recompensa, terminado):
        estado_anterior_discreto = discretize_state(estado_anterior, self.bins)
        estado_siguiente_discreto = discretize_state(estado_siguiente, self.bins)

        self.visitas[estado_anterior_discreto][accion] += 1

        """Updates the Q-value of an action."""
        future_q_value = (not terminado) * np.max(self.q_table[estado_siguiente_discreto])
        temporal_difference = (
            recompensa + self.discount_factor * future_q_value - self.q_table[estado_anterior_discreto][accion]
        )

        lr = 1 / self.visitas[estado_anterior_discreto][accion]

        self.q_table[estado_anterior_discreto][accion] = (
            self.q_table[estado_anterior_discreto][accion] + lr * temporal_difference
        )
        self.training_error.append(temporal_difference)

    def fin_episodio(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

Y ejecutar con el muchos episodios:

In [11]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)
# Advertencia: este bloque es un loop infinito si el agente se deja sin implementar
from tqdm import tqdm 
entorno = gym.make('LunarLander-v2').env
num_episodios = 100000
max_accion = entorno.action_space.n
initial_epsilon = 1.0
epsilon_decay = initial_epsilon / (num_episodios / 2)
final_epsilon = 0.05
discount_factor = 0.95

agente = AgenteRL(max_accion = max_accion, bins = bins, initial_epsilon = initial_epsilon, epsilon_decay = epsilon_decay, final_epsilon = final_epsilon, discount_factor = discount_factor)
exitos = 0
recompensa_episodios = []
with tqdm(total=num_episodios) as pbar:
    for i in range(num_episodios):
        recompensa = ejecutar_episodio(agente)
        if i % 1000 == 0:
            ultimas_1000_recompensas = recompensa_episodios[-1000:]
            promedio_ultimas_1000 = np.mean(ultimas_1000_recompensas)
            print(f" Se obtuvo {promedio_ultimas_1000} de recompensa, en promedio en los últimos 1000 episodios")
        if (recompensa >= 200):
            exitos += 1
        recompensa_episodios += [recompensa]
        pbar.set_postfix(recompensa=recompensa, promedio=numpy.mean(recompensa_episodios))
        pbar.update(1)
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

  0%|          | 0/10000 [00:00<?, ?it/s]/home/julio/.local/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/julio/.local/lib/python3.10/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
  0%|          | 9/10000 [00:00<02:36, 63.68it/s, promedio=-177, recompensa=-414] 

 Se obtuvo nan de recompensa, en promedio en los últimos 1000 episodios


 10%|█         | 1013/10000 [00:16<02:21, 63.42it/s, promedio=-162, recompensa=-74.8]

 Se obtuvo -163.12802467743523 de recompensa, en promedio en los últimos 1000 episodios


 20%|██        | 2011/10000 [00:35<02:32, 52.23it/s, promedio=-155, recompensa=-109]  

...

 Se obtuvo -62.106232901033714 de recompensa, en promedio en los últimos 1000 episodios


 90%|█████████ | 9003/10000 [05:00<00:51, 19.33it/s, promedio=-109, recompensa=-47.5]  

 Se obtuvo -48.36376092504068 de recompensa, en promedio en los últimos 1000 episodios


100%|██████████| 10000/10000 [05:45<00:00, 28.94it/s, promedio=-101, recompensa=-94.7]


Tasa de éxito: 0.0351. Se obtuvo -101.47377887979772 de recompensa, en promedio


In [15]:
# Nota: hay que transformar esta celda en código para ejecutar (Esc + y)
# Advertencia: este bloque es un loop infinito si el agente se deja sin implementar

entorno = gym.make('LunarLander-v2').env
exitos = 0
recompensa_episodios = []
num_episodios = 1000
for i in range(num_episodios):
    recompensa = ejecutar_episodio(agente,aprender=False)
    # Los episodios se consideran exitosos si se obutvo 200 o más de recompensa total
    if (recompensa >= 200):
        print(f"Episodio {i} exitoso, recompensa: {recompensa}")
        exitos += 1
    recompensa_episodios += [recompensa]
print(f"Tasa de éxito: {exitos / num_episodios}. Se obtuvo {numpy.mean(recompensa_episodios)} de recompensa, en promedio")

Episodio 6 exitoso, recompensa: 268.13116411594643
Episodio 14 exitoso, recompensa: 249.33242900160633
Episodio 16 exitoso, recompensa: 301.0171947924978
Episodio 19 exitoso, recompensa: 274.30589432994543
Episodio 25 exitoso, recompensa: 212.29565155177
Episodio 26 exitoso, recompensa: 260.4641392409631
Episodio 27 exitoso, recompensa: 289.53369501354143
Episodio 28 exitoso, recompensa: 228.50444231573678
Episodio 30 exitoso, recompensa: 251.42573131790834
Episodio 31 exitoso, recompensa: 273.96806238209587
Episodio 32 exitoso, recompensa: 204.43916637090666
Episodio 38 exitoso, recompensa: 254.1531165100468
Episodio 42 exitoso, recompensa: 255.4817674462194
Episodio 44 exitoso, recompensa: 244.32230250852126
Episodio 45 exitoso, recompensa: 251.09426614509
Episodio 46 exitoso, recompensa: 287.4614073373566
Episodio 51 exitoso, recompensa: 267.4499579630059
Episodio 55 exitoso, recompensa: 219.9848000729709
Episodio 63 exitoso, recompensa: 231.71184796459937
Episodio 72 exitoso, recom

In [1]:
# Estado:
# (x, y, x_vel, y_vel, theta, theta_vel, pie_izq_en_contacto, pie_derecho_en_contacto)
# Nave cayendo rapidamente hacia abajo sin rotacion en la matriz Q
estado = discretize_state([0, 1.5, 0, -5, 0, 0, 0, 0], bins)
print(agente.q_table[estado])
# Nave cayendo rapidamente hacia abajo sin rotacion en la matriz Q
estado = discretize_state([0, 1.5, 0, -4, 0, 0, 0, 0], bins)
print(agente.q_table[estado])
# Nave cayendo rapidamente hacia abajo sin rotacion en la matriz Q
estado = discretize_state([0, 0.5, 0, -2, 0, 0, 0, 0], bins)
print(agente.q_table[estado])
estado_final = discretize_state([0, 0, 0, 0, 0, 0, 1, 1], bins)
print(agente.q_table[estado_final])

estado_2 = discretize_state([0, 0, 0, 0, 0, 0, 0, 1], bins)
print(agente.q_table[estado_2])

estado_3 = discretize_state([0, 0, 0, 0, 0, 0, 1, 0], bins)
print(agente.q_table[estado_3])

# Aterriza pero no en el centro der
estado_4 = discretize_state([1.5, 0, 0, 0, 0, 0, 0, 0], bins)
print(agente.q_table[estado_4])

# Aterriza en el centro izq
estado_5 = discretize_state([-1.5, 0, 0, 0, 0, 0, 0, 0], bins)
print(agente.q_table[estado_5])

# Estado inicial
estado_6 = discretize_state([0, 1.5, 0, 0, 0, 0, 0, 0], bins)
print(agente.q_table[estado_6])

NameError: name 'discretize_state' is not defined

Analizar los resultados de la ejecución anterior, incluyendo:
 * Un análisis de los parámetros utilizados en el algoritmo (aprendizaje, política de exploración)
 * Un análisis de algunos 'cortes' de la matriz Q y la política (p.e. qué hace la nave cuando está cayendo rápidamente hacia abajo, sin rotación)
 * Un análisis de la evolución de la recompensa promedio
 * Un análisis de los casos de éxito
 * Un análisis de los casos en el que el agente falla
 * Qué limitante del agente de RL les parece que afecta más negativamente su desempeño. Cómo lo mejorarían? 

# Análisis de resultados

En la siguiente sección estaremos realizando un análisis de los resultados obtenidos tras la implementación y el entrenamiento de un agente de RL basado en Q Learning. En esta sección explicaremos cómo construimos el agente, en qué nos basamos, qué parámetros tuvimos en cuenta y por qué. También analizaremos algunas situaciones particulares del agente para ver su comportamiento en detalle, algunos casos de éxito y de falla, cómo evolucionó la recompensa promedio a medida que avanzaba el entrenamiento y qué limitantes vemos en el agente y cómo se podrían mitigar.


## Análisis de implementación y decisiones de diseño

A la hora de construir nuestro agente RL basado en Q Learning, nos basamos fuertemente en el teórico del curso, utilizando la metodología vista en clase, y también en la documentación de gymnasium, en la cual explican, sobre otro ejemplo, como entrenarlo con Q Learning utilizando la librería. El link a la documentación utilizada es el siguiente: https://gymnasium.farama.org/tutorials/training_agents/blackjack_tutorial/.

### Estrategia de acción a tomar

Lo primero que decidimos sobre el modelo es cómo debiamos elegir una acción, dado que si siempre elegimos la mejor acción nos perdemos de encontrar acciones que no fueron descubiertas previamente, por lo que hay que encontrar un balance entre tomar decisiones azarosas y las mejores decisiones. Ante esto teníamos 2 opciones: 
- Soft Max: En esta estrategia, se asignan probabilidades a las diferentes acciones usando una función. Las acciones con valores de Q más altos tendrán mayores probabilidades de ser seleccionadas, pero incluso las acciones con valores menores todavía tienen una probabilidad no nula de ser escogidas, lo que genera que se suela elegir las acciones con valores de Q más altos, pero que con cierta probabilidad no nula se tomen otras acciones que pueden resultar beneficiosas.
- Epsilon-Greedy: En esta estrategia, se toma un valor epsilon, que viene dado como parámetro, y se toma la mejor acción (mayor valor de la tabla Q para ese estado) con probabilidad 1-epsilon, y una decisión azarosa con probabilidad epsilon. De esta forma se asegura que tomemos acciones azarosas e informadas, lo que contribuye a un mejor entrenamiento.
- Epsilon-Greedy con decaimiento: Esta estrategia es una variante de la estrategia Epsilon-Greedy que aparte del valor epsilon toma un valor de epsilon minimo y un valor de decaimiento de epsilon. Por cada episodio que pasa se reduce el epsilon restandole el valor de decaimiento, hasta que en algún episodio se llega al valor mínimo de epsilon, el cual se utiliza durante el resto de entrenamiento. Esto lo que genera es que al principio del entrenamiento tomemos más decisiones azarosas, ya que no tenemos tanto conocimiento del entorno aún, y a medida que pasa el tiempo y se asume que tenemos más información y podemos tomar más decisiones informadas, la probabilidad de tomar decisiones azarosas se reduce y en su lugar se toman decisiones informadas con respecto a los valores de la tabla Q.

Ante estas tres opciones decidimos inclinarnos por la tercera opción: Epsilon-Greedy con decaimiento. Nos pareció una estrategia coherente, un poco mejor que el Epsilon-Greedy sin esta variante, y creímos que en términos de implementación no era muy compleja. Decidimos probar esta estrategia y en caso de notar un bajo desempeño, cambiar a Soft Max. Cómo se verá más adelante en este informe, tras ver que los resultados fueron óptimos, se mantuvo la decisión.

### Hiperparámetros a utilizar y valores utilizados

#### Epsilon, Epsilon Mínimo y Decaimiento de Epsilon

Tras la decisión de la estrategia a utilizar, debíamos decidir qué valores utilizar para dichos parámetros (epsilon, decaimiento de epsilon y epsilon mínimo). Para el epsilon se tomó la decisión del valor de 1, ya que entendimos que cuando recién se comienza el entrenamiento queremos explorar lo más que se pueda, ya que no tenemos ningún tipo de información del entorno aún, por lo que la probabilidad conviene que sea 1, y que luego esta vaya disminuyendo linealmente. Cómo epsilon mínimo tomamos el valor de 0.05. Esta decisión se baso mayormente en experiencia. Sabíamos que queríamos un valor bajo de epsilon mínimo ya que una vez que tengamos la tabla Q bastante llena, queremos tomar decisiones informadas la mayor parte de las veces. Al principio probamos con un valor de 0.1, de 0.2, de 0.05, y así fuimos probando con distintos valores, llegando a que los mayores resultados se daban con el valor de 0.05, aunque la diferencia no era tanta, sí había una diferencia.
Por último, debimos decidir sobre el valor del decaimiento de epsilon.
Sobre este valor nos encontramos con un tema a la hora de la implementación, y fue que al principio utilizamos el decaimiento de epsilon como factor que se le multiplicaba al valor de epsilon hasta llegar al epsilon mínimo. Por lo que poniamos un valor fijo alto de decaimiento de epsilon (ej: 0.995). Vimos que los resultados no eran buenos y decidimos probar con valores paramétricos, utilizando la cantidad de episodios para decidir sobre el valor. Para esto se nos ocurrió la fórmula: decaimiento de epsilon = (epsilon - epsilon min) ^ (1/episodios), con la idea de que esto haría que el epsilon fuera decayendo y en el último episodio se llegara al valor mínimo de epsilon.
Esta opción tampoco nos trajo buenos resultados, por lo que tras revisar la documentación brindada por gymnasium, vimos que estaba bien poner un valor parámetrico en función de la cantidad de episodios, pero que la disminución del epsilon con el factor de decaimiento era a través de una resta. Epsilon = Epsilon - decaimiento_epsilon. En este caso vimos que en la documentación utilizaban un valor de decaimiento descripto por la fórmula: decaimiento_epsilon = epsilon / (num_episodios / 2). Tras utilizar esta fórmula nuestros resultados mejoraron bastante. Probamos luego variar el valor de 2 en la fórmula por distintos múltiplos de 2, ya que entendimos que por cómo es su funcionamiento, cuanto más grande dicho valor, durante menos tiempo se mantiene el epsilon en un número alto, pero vimos que la fórmula detallada por la documentación fue la que nos dió mejores resultados en la práctica.

#### Tasa de aprendizaje en ambiente no determinista

Dado que nos encontramos en el problema de Lunar Lander, este es un problema continuo, donde los estados son los ejes horizontal, vertical y de rotación, sus velocidades y 2 booleanos que indican si las patas de la nave están en contacto con el suelo o no. Por más de que en un principio discretizamos el estado en una cantidad de bins, esto no quita que el problema deje de ser continuo, por lo que al trabajar con un problema continuo, nos va a suceder muchas veces que al tomar una misma acción desde un mismo estado caigamos en estados distintos. Esto tiene un problema si no lo tomamos en cuenta, y es que a la hora de llenar la tabla Q con los valores de recompensa, las recompensas no van a ser las mismas desde un estado y una acción ya que los resultados posteriores dependen del estado en el que se caiga posteriormente, y estos varían cada vez que los tomamos. 
Para esto se utiliza el hiperparámetro tasa de aprendizaje.






## Análisis de "cortes" de la matriz Q y la política

## Análisis de evolución de recompensa promedio

## Análisis de casos (Éxitos y Fallos)

## Limitantes del agente

In [14]:
# Analizar los resultados aqui
